In [1]:
import tensorflow

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지

from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import DenseNet121 #Densenet121 모델 사용
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [2]:
model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) #채널이 무조건 3개여야 하며 크기는 최소 32 이상

x=model_d.output

x= GlobalAveragePooling2D()(x) #전역 평균 풀링 레이어 추가
x= BatchNormalization()(x) #배치 정규화 레이어
x= Dropout(0.5)(x)
#Fully Connected 레이어 추가
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x) #과적합 감소용 드롭아웃 레이어

preds=Dense(7,activation='softmax')(x) #FC-layer. 클래스가 7개이므로 softmax 7개 설정

In [3]:
model=Model(inputs=model_d.input,outputs=preds)
model.summary() #모델 요약

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [4]:
#Freeze
for layer in model.layers[:-8]:
    layer.trainable=False
    
for layer in model.layers[-8:]:
    layer.trainable=True

In [5]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy']) #모델 생성
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
______________________________________________________________________________________________

__________________________________________________________________________________________________
conv5_block3_concat (Concatenat (None, 4, 4, 608)    0           conv5_block2_concat[0][0]        
                                                                 conv5_block3_2_conv[0][0]        
__________________________________________________________________________________________________
conv5_block4_0_bn (BatchNormali (None, 4, 4, 608)    2432        conv5_block3_concat[0][0]        
__________________________________________________________________________________________________
conv5_block4_0_relu (Activation (None, 4, 4, 608)    0           conv5_block4_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block4_1_conv (Conv2D)    (None, 4, 4, 128)    77824       conv5_block4_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_bloc

In [6]:
data=[] #데이터
labels=[] #라벨
random.seed(42)
imagePaths = sorted(list(os.listdir("emotion3/"))) #데이터셋 경로
random.shuffle(imagePaths) #셔플
print(imagePaths)
#이미지 읽기 및 라벨링
for img in imagePaths:
    path=sorted(list(os.listdir("emotion3/"+img)))
    for i in path:
        image = cv2.imread("emotion3/"+img+'/'+i) #이미지 읽기
        image = cv2.resize(image, (128,128)) #이미지 사이즈 편집
        image = img_to_array(image) #이미지 배열화
        data.append(image) #data 배열에 데이터 추가
        l = label = img
        labels.append(l) #labels 배열에 데이터 추가

['disgusted', 'happy', 'neutral', 'fearful', 'surprised', 'angry', 'sad']


In [7]:
data = np.array(data, dtype="float32") / 255.0 #데이터 타입 변경 및 numpy 배열화
labels = np.array(labels) #라벨 numpy 배열화
mlb = LabelBinarizer() #데이터 전처리(원핫인코딩)
labels = mlb.fit_transform(labels) #데이터에 대해서 fit 작업과 transform 작업을 적용해주는 것
print(labels[0])

[0 1 0 0 0 0 0]


In [8]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.3,random_state=42) #train용과 test용으로 데이터셋 분리. test 30%
print(xtrain.shape, xtest.shape)

(20096, 128, 128, 3) (8613, 128, 128, 3)


In [9]:
#검증 손실에 변화가 없는 경우 학습률을 감소시켜 도움을 줌
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3) 
checkpoint = ModelCheckpoint('Densenet121_revised4.h5', verbose=1, save_best_only=True) #체크포인트를 두고 모델 저장

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)


datagen.fit(xtrain)
# 모델 학습
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=128),
               steps_per_epoch=xtrain.shape[0] //128,
               epochs=50,
               verbose=2,#verbose는 '세대 당 한 라인'을 의미
               callbacks=[anne, checkpoint], #학습과 검증 과정에서 적용할 콜백 리스트
               validation_data=(xtrain, ytrain)) #steps_per_epoch일 때 사용하는 것으로 샘플 배치 데이터 수

Epoch 1/50
157/157 - 96s - loss: 2.0971 - accuracy: 0.3408 - val_loss: 1.4670 - val_accuracy: 0.4476

Epoch 00001: val_loss improved from inf to 1.46698, saving model to Densenet121_revised4.h5
Epoch 2/50
157/157 - 76s - loss: 1.6440 - accuracy: 0.3961 - val_loss: 1.3893 - val_accuracy: 0.4787

Epoch 00002: val_loss improved from 1.46698 to 1.38929, saving model to Densenet121_revised4.h5
Epoch 3/50
157/157 - 77s - loss: 1.5396 - accuracy: 0.4255 - val_loss: 1.3443 - val_accuracy: 0.4872

Epoch 00003: val_loss improved from 1.38929 to 1.34432, saving model to Densenet121_revised4.h5
Epoch 4/50
157/157 - 78s - loss: 1.4777 - accuracy: 0.4366 - val_loss: 1.3053 - val_accuracy: 0.5114

Epoch 00004: val_loss improved from 1.34432 to 1.30527, saving model to Densenet121_revised4.h5
Epoch 5/50
157/157 - 77s - loss: 1.4493 - accuracy: 0.4448 - val_loss: 1.2819 - val_accuracy: 0.5125

Epoch 00005: val_loss improved from 1.30527 to 1.28192, saving model to Densenet121_revised4.h5
Epoch 6/50
157


Epoch 00043: val_loss improved from 0.73649 to 0.72684, saving model to Densenet121_revised4.h5
Epoch 44/50
157/157 - 79s - loss: 1.0670 - accuracy: 0.6035 - val_loss: 0.7199 - val_accuracy: 0.7711

Epoch 00044: val_loss improved from 0.72684 to 0.71992, saving model to Densenet121_revised4.h5
Epoch 45/50
157/157 - 79s - loss: 1.0484 - accuracy: 0.6063 - val_loss: 0.7037 - val_accuracy: 0.7694

Epoch 00045: val_loss improved from 0.71992 to 0.70368, saving model to Densenet121_revised4.h5
Epoch 46/50
157/157 - 79s - loss: 1.0545 - accuracy: 0.6063 - val_loss: 0.6929 - val_accuracy: 0.7777

Epoch 00046: val_loss improved from 0.70368 to 0.69286, saving model to Densenet121_revised4.h5
Epoch 47/50
157/157 - 79s - loss: 1.0412 - accuracy: 0.6091 - val_loss: 0.6901 - val_accuracy: 0.7682

Epoch 00047: val_loss improved from 0.69286 to 0.69009, saving model to Densenet121_revised4.h5
Epoch 48/50
157/157 - 78s - loss: 1.0338 - accuracy: 0.6129 - val_loss: 0.6727 - val_accuracy: 0.7779

Epoc

In [10]:
#완성도 확인
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]): #가장 큰 값을 찾아 인덱스 반환
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

Total-test-data; 8613 	accurately-predicted-data: 4822 	 wrongly-predicted-data:  3791
Accuracy: 55.985 %
